In [ ]:
# Libraries

import pandas as pd
import json     
import snowflake.connector
import datetime

#----------CONNECTION TO SNOWFLAKE-------------
SF_ACCOUNT = 'gfa04036.us-east-1'
SF_WH = 'TRANSFORMING'
SF_USERNAME = 'DBT_USER'
SF_PASSWORD = '2C>`8Q!8y*Sz]h/):Xxy&WNJv'

# Connecting to Snowflake using the default authenticator
ctx = snowflake.connector.connect(
  user=SF_USERNAME,    #username,
  password=SF_PASSWORD,    #password,
  account=SF_ACCOUNT,
  warehouse=SF_WH,
  database='ANALYTICS',
  schema='PROD_STAGING'
)

cur=ctx.cursor()